## Part 5 and 6

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('breast_cancer_data.csv')

df = df.drop(columns='id')

X = df.drop(columns='diagnosis')
y = df['diagnosis'] == 'M'

y.value_counts()

diagnosis
False    357
True     212
Name: count, dtype: int64

5. Use GridSearchCV to tune the parameter of each of the above models. Can
you obtain better results in this step for any of the models? Discuss your
observations.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/home/franciscoyira/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

Defining the parameter grids to tune each model:

In [7]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()

# Defining a dictionary with all the models
models = {
    'Logistic Regression': lr,
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SGD': SGDClassifier(),
    'SVM': SVC()
}

# Defining a parameter grid for each model
param_grid = {
    # Regularisation strength
    'Logistic Regression': {'C': [0.01, 0.1, 1, 10, 100]},

    # Max depth, associated with flexibility of the model
    'Decision Tree': {'max_depth': [None, 10, 20, 30, 40, 50]},

    # N_estimators (number of trees to use)
    'Random Forest': {'n_estimators': [10, 50, 100, 200]},

    # alpha: multiplier of the regularisaiton term
    'SGD': {'alpha': [0.0001, 0.001, 0.01, 0.1]},
    
    'SVM': {
        # Regularisation parameter
        'C': [0.01, 0.1, 1],
        # Kernel to use
        'kernel': ['linear', 'rbf']}
}

After having defined the parameter grid, we can run a loop across the models to tune each of them, and take a look at the resuls:

In [ ]:
# Performing Grid Search for each model
for model_name, model in models.items():
    # Running the Grid Search (fitting it to the data)
    print(f"Working on model: {model_name}")
    grid_search = GridSearchCV(model, param_grid[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Best parameters and best score
    print(f"Best Parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best Score for {model_name}: {grid_search.best_score_}")

    # Evaluate on the test set
    y_pred = grid_search.predict(X_test)
    print(f"Classification Report for {model_name}:\n{classification_report(y_test, y_pred)}")
    print(f"Accuracy for {model_name}: {accuracy_score(y_test, y_pred)}\n")




6. Randomly (or based on certain hypothesis) remove some features and
re-evaluate the models. Document your observations with respect to models
performances.

In [12]:
grid_features_to_remove = [
  ['area_mean'],
  ['radious_mean', 'texture_mean'],
  None]

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureDropper(BaseEstimator, TransformerMixin):
    def __init__(self, features_to_drop=None):
        # features_to_drop can be a list of feature names or None
        self.features_to_drop = features_to_drop
    
    def fit(self, X, y=None):
        # No fitting process required as we're only dropping features
        return self
    
    def transform(self, X):
        # Make a copy of the input DataFrame to avoid altering original data
        X_modified = X.copy()
        
        # If features_to_drop is not None and is a list, drop the specified features
        if self.features_to_drop is not None:
            X_modified = X_modified.drop(self.features_to_drop, axis=1, errors='ignore')
        
        return X_modified


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier  # Example estimator

pipeline = Pipeline([
    ('feature_dropper', FeatureDropper(features_to_drop=[None])),  # Initially set to not drop
    ('classifier', RandomForestClassifier())
])

param_grid = {
    'feature_dropper__features_to_drop': grid_features_to_remove
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('feature_dropper',
                                        FeatureDropper(features_to_drop=[None])),
                                       ('scaler', StandardScaler()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'feature_dropper__features_to_drop': [['area_mean'],
                                                               ['radious_mean',
                                                                'texture_mean'],
                                                               None]})

In [15]:
print(grid_search.best_params_)

{'feature_dropper__features_to_drop': None}


In [16]:
grid_search.param_grid

{'feature_dropper__features_to_drop': [['area_mean'],
  ['radious_mean', 'texture_mean'],
  None]}

In [20]:
results = pd.DataFrame(grid_search.cv_results_)


relevant_columns = ['mean_test_score', 'params']

display_results = results[relevant_columns].sort_values(by='mean_test_score', ascending=False)
display_results


,mean_test_score,params
2,0.953846,{'feature_dropper__features_to_drop': None}
0,0.949451,{'feature_dropper__features_to_drop': ['area_m...
1,0.947253,{'feature_dropper__features_to_drop': ['radiou...
